## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [6]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

In [27]:
#範例沒有的random_search
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

In [7]:
iris = datasets.load_iris()

#切割訓練驗證資料
x_train,x_test,y_train,y_test = train_test_split(iris.data,iris.target,test_size=0.25,random_state=4)
gb = GradientBoostingClassifier()

In [15]:
gb.fit(x_train,y_train)
y_pred = gb.predict(x_test)
print("ACC Rate = %f" %metrics.accuracy_score(y_test,y_pred))

ACC Rate = 0.973684


In [17]:
#嘗試使用GridSearch窮舉法

# 設定要訓練的超參數組合
n_estimators = [100, 200, 300]
max_depth = [1, 3, 5]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(gb, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 預設會跑 3-fold cross-validadtion，總共 9 種參數組合，總共要 train 27 次模型

# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 9 candidates, totalling 27 fits


/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Best Accuracy: -0.026786 using {'max_depth': 5, 'n_estimators': 100}


[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    3.5s finished
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [23]:
#用最佳結果建模  (結果相同)
gb2 = GradientBoostingClassifier(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])
gb2.fit(x_train,y_train)
y_pred = gb2.predict(x_test)
print("ACC Rate = %f" %metrics.accuracy_score(y_test,y_pred))

ACC Rate = 0.973684


In [30]:
#如果用random呢?

param_dist = {
        'n_estimators':range(80,200,4),
        'max_depth':range(2,15,1),
        'learning_rate':np.linspace(0.01,2,20),
        'subsample':np.linspace(0.7,0.9,20)
#         'colsample_bytree':np.linspace(0.5,0.98,10),
#         'min_child_weight':range(1,9,1)
        }

rand_search = RandomizedSearchCV(gb,param_dist, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)


# 開始搜尋最佳參數
rand_result = rand_search.fit(x_train, y_train)

# 預設會跑 3-fold cross-validadtion，總共 9 種參數組合，總共要 train 27 次模型

# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (rand_result.best_score_, rand_result.best_params_))

Fitting 3 folds for each of 10 candidates, totalling 30 fits


/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    2.5s finished
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best Accuracy: -0.017857 using {'subsample': 0.8368421052631578, 'n_estimators': 192, 'max_depth': 14, 'learning_rate': 0.7431578947368421}


In [33]:
#用最佳結果建模  (結果相同)
gb2 = GradientBoostingClassifier(max_depth=rand_result.best_params_['max_depth'],
                                 n_estimators=rand_result.best_params_['n_estimators'],
                                 subsample = rand_result.best_params_['subsample'],
                                 learning_rate = rand_result.best_params_['learning_rate']
                                )
gb2.fit(x_train,y_train)
y_pred = gb2.predict(x_test)
print("ACC Rate = %f" %metrics.accuracy_score(y_test,y_pred))

ACC Rate = 0.973684
